# RE RE RE RE FEATURE ENGINEERING OF DATA - adding merchant details

In [1]:
import numpy as np
import pandas as panda
from dateutil import relativedelta
import datetime

In [2]:
train_data_path = 'all_new/train_2.csv'
test_data_path = 'all_new/test_2.csv'
new_transaction_path = 'C:/Users/somak/Documents/somak_python/real_world_use_cases_1/supervised/regression/kaggle/elo/all/historical_transactions.csv'
historical_transaction_path = 'all_new/historical_transactions.csv'
new_treated_transaction_path = 'all_new/treated_new_transaction.csv'
treated_historical_transaction_path = 'all_new/treated_historical_transactions.csv'
merchants_data_path = 'all_new/agg_merchants_1.csv'



In [3]:
merchant_data = panda.read_csv(merchants_data_path)
# merchant_data.columns = list(map(lambda x: x if x.startswith('merchant_') else 'merchant_'+x, merchant_data.columns.tolist()))
# ['merchant_'+i for i in merchant_data.columns.tolist() if not i.startswith('merchant_')]
merchant_data.head().T

,0,1,2,3,4
merchant_category_id,-1,2,2,2,2
state_id,9,1,1,2,2
city_id,231,-1,246,-1,76
subsector_id,27,20,20,20,20
category_1,N,N,N,N,N
merchant_group_id_nunique,1,4,4,4,15
merchant_id_nunique,1,4,4,4,15
merchant_id_most_frequent_merchant_id,M_ID_a2cfe4149a,M_ID_eeba19250e,M_ID_628321e5da,M_ID_0284e321ff,M_ID_6ef27dc5bd
numerical_1_sum,-0.0475558,0.0378198,0.0279049,0.672374,-0.148187
numerical_1_mean,-0.0475558,0.00945495,0.00697623,0.168093,-0.00987911


In [4]:
## deres nothing else we can do now...with this data
## lets look at new transactions first , since it is a smaller data sets

new_transaction_data = panda.read_csv(new_transaction_path)
new_transaction_data_types = new_transaction_data.dtypes
new_transaction_column_types = {}
for index, column_name in enumerate(new_transaction_data.columns.tolist()):
    new_transaction_column_types[column_name] = new_transaction_data_types[index]
del new_transaction_data
new_transaction_data = panda.read_csv(new_transaction_path, dtype = new_transaction_column_types)
new_transaction_data.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37
1,Y,C_ID_4e6213e9bc,88,N,0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16
2,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37
3,Y,C_ID_4e6213e9bc,88,N,0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34
4,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1.0,16,37


In [5]:
new_transaction_data[new_transaction_data.merchant_category_id==-1].shape

(2252, 14)

In [6]:
merchant_data[merchant_data.merchant_category_id==-1]

,merchant_category_id,state_id,city_id,subsector_id,category_1,merchant_group_id_nunique,merchant_id_nunique,merchant_id_most_frequent_merchant_id,numerical_1_sum,numerical_1_mean,...,most_sales_range_E_sum,most_sales_range_D_sum,most_sales_range_C_sum,most_sales_range_B_sum,most_sales_range_A_sum,most_recent_purchases_range_E_sum,most_recent_purchases_range_D_sum,most_recent_purchases_range_C_sum,most_recent_purchases_range_B_sum,most_recent_purchases_range_A_sum
0,-1,9,231,27,N,1,1,M_ID_a2cfe4149a,-0.047556,-0.047556,...,1,0,0,0,0,1,0,0,0,0


In [7]:
def prep_for_aggregate(table):
    
    table['authorized_flag'] = table.authorized_flag.map({'Y':1,'N':0})
    table['category_1'] = table.category_1.map({'Y':1, 'N':0})
    most_appearing = table.category_2.value_counts().idxmax()
    table.category_2.fillna(value= most_appearing, inplace=True)
    table['enc_category_2_1'] = (table.category_2==1) + 0
    table['enc_category_2_2'] = (table.category_2==2) + 0
    table['enc_category_2_3'] = (table.category_2==3) + 0
    table['enc_category_2_4'] = (table.category_2==4) + 0
    table['enc_category_2_5'] = (table.category_2==5) + 0

    table.drop(columns = ['category_2'], inplace=True)
    most_appearing = table.category_3.value_counts().idxmax()
    table.category_3.fillna(value = most_appearing, inplace = True)
    table['enc_category_3_A'] = (table.category_3=='A') +0
    table['enc_category_3_B'] = (table.category_3=='B') +0
    table['enc_category_3_C'] = (table.category_3=='C') +0
    table.drop(columns = ['category_3'], inplace = True)
    table['enc_purchase_date'] = panda.DatetimeIndex(table.purchase_date).astype(np.int64)*1e-9
    table.drop(columns = ['purchase_date'], inplace = True)
    most_appearing = table.city_id.value_counts().idxmax()
    table.city_id.replace({-1:most_appearing}, inplace = True)
    most_appearing = table.state_id.value_counts().idxmax()
    table.state_id.replace({-1:most_appearing}, inplace = True)
    most_appearing = table.subsector_id.value_counts().idxmax()
    table.subsector_id.replace({-1:most_appearing}, inplace = True)
    
    

In [8]:
prep_for_aggregate(new_transaction_data)
new_transaction_data.head()

,authorized_flag,card_id,city_id,category_1,installments,merchant_category_id,merchant_id,month_lag,purchase_amount,state_id,subsector_id,enc_category_2_1,enc_category_2_2,enc_category_2_3,enc_category_2_4,enc_category_2_5,enc_category_3_A,enc_category_3_B,enc_category_3_C,enc_purchase_date
0,1,C_ID_4e6213e9bc,88,0,0,80,M_ID_e020e9b302,-8,-0.703331,16,37,1,0,0,0,0,1,0,0,1.498405e+09
1,1,C_ID_4e6213e9bc,88,0,0,367,M_ID_86ec983688,-7,-0.733128,16,16,1,0,0,0,0,1,0,0,1.500121e+09
2,1,C_ID_4e6213e9bc,88,0,0,80,M_ID_979ed661fc,-6,-0.720386,16,37,1,0,0,0,0,1,0,0,1.502316e+09
3,1,C_ID_4e6213e9bc,88,0,0,560,M_ID_e6d5ae8ea6,-5,-0.735352,16,34,1,0,0,0,0,1,0,0,1.504347e+09
4,1,C_ID_4e6213e9bc,88,0,0,80,M_ID_e020e9b302,-11,-0.722865,16,37,1,0,0,0,0,1,0,0,1.489108e+09


In [9]:
## we will take the 
## sum,mean,max of purchase amount, 
## sum,mean,max of month lag by card_id
## mean of authorized_flag,
## num of authorized ==1
## num of authorized ==0
## sum, mean, max of installments
## mean of category_1
## sum of enc_category_2* and enc_category_3*
from collections import Counter

def count_authorized_one(x):
    return x.eq(1).sum()
def count_authorized_zero(x):
    return x.eq(0).sum()
def count_category_1_one(x):
    return x.eq(1).sum()
def count_category_1_zero(x):
    return x.eq(0).sum()

def count_of_unique_cities(x):
    return x.nunique()

def count_of_unique_subsector_id(x):
    return x.nunique()

def count_of_unique_state_id(x):
    return x.nunique()

def most_frequented_merchant(x):
    c = Counter(x)
    return c.most_common()[0][0]

In [13]:

# aggregate = {
    
# 'purchase_amount' : {'new_purchase_amount_sum':'sum','new_purchase_amount_mean': 'mean','new_purchase_amount_max':'max'},
# 'month_lag' : {'new_month_lag_sum':'sum', 'new_month_lag_mean':'mean','new_month_lag_max':'max'} ,
# 'enc_category_2_1':{'new_category_2_1_sum':'sum'},
# 'enc_category_2_2':{'new_category_2_2_sum':'sum'},
# 'enc_category_2_3':{'new_category_2_3_sum':'sum'},
# 'enc_category_2_4':{'new_category_2_4_sum':'sum'},
# 'enc_category_2_5':{'new_category_2_5_sum':'sum'},    
# 'enc_category_3_A':{'new_category_3_A_sum':'sum'},
# 'enc_category_3_B':{'new_category_3_B_sum':'sum'},
# 'enc_category_3_C':{'new_category_3_C_sum':'sum'},
# 'installments': {'new_installments_sum':'sum','new_installments_mean':'mean','new_installments_max':'max'},
# 'authorized_flag': {'new_authorized_flag_mean':'mean', 'new_authorized_flag_count_authorized_one':count_authorized_one, 'new_authorized_flag_count_authorized_zero':count_authorized_zero},
# 'category_1' :{'new_category_1_mean':'mean', 'new_category_1_count_category_1_one':count_category_1_one, 'new_category_1_count_category_1_zero':count_category_1_zero},
# 'enc_purchase_date' : {'new_purchase_date_mean':'mean','new_purchase_date_min':'min','new_purchase_date_max':'max'},
# 'city_id' : {'new_city_id_nunique':count_of_unique_cities},
# 'state_id': {'new_state_id_nunique':count_of_unique_state_id},
# 'subsector_id' : {'new_subsector_id_nunique':count_of_unique_subsector_id}
    
# }



aggregate = {
    
'purchase_amount' : ['sum','mean','max'],
'month_lag' : ['sum','mean','max'] ,
'enc_category_2_1':['sum'],
'enc_category_2_2': ['sum'],
'enc_category_2_3': ['sum'],
'enc_category_2_4': ['sum'],
'enc_category_2_5': ['sum'],    
'enc_category_3_A': ['sum'],
'enc_category_3_B': ['sum'],
'enc_category_3_C': ['sum'],
'installments': ['sum','mean','max'],
'authorized_flag': ['mean', count_authorized_one, count_authorized_zero],
'category_1' :['mean', count_category_1_one, count_category_1_zero],
'enc_purchase_date' : ['mean','min','max',np.ptp],
'city_id' : [count_of_unique_cities],
'state_id': [count_of_unique_state_id],
'subsector_id' : [count_of_unique_subsector_id],
# 'merchant_id' : [most_frequented_merchant]
    
}


In [14]:
group_by = new_transaction_data.groupby(['card_id'])

In [15]:
len(new_transaction_data)

29112361

In [16]:
t = group_by.agg(aggregate)

In [17]:
t.head()

purchase_amount                     month_lag                \
                            sum      mean       max       sum      mean max   
card_id                                                                       
C_ID_00007093c1      -76.845041 -0.515739  1.507069      -872 -5.852349   0   
C_ID_0001238066      -72.447201 -0.589002  0.768095      -223 -1.813008   0   
C_ID_0001506ef0      -34.601879 -0.524271  1.493545      -319 -4.833333   0   
C_ID_0001793786      -36.786013 -0.170306  4.554145      -719 -3.328704   0   
C_ID_000183fdda      -68.837938 -0.478041  2.764788      -353 -2.451389   0   

                enc_category_2_1 enc_category_2_2 enc_category_2_3  \
                             sum              sum              sum   
card_id                                                              
C_ID_00007093c1               28                0              120   
C_ID_0001238066              103                0                0   
C_ID_0001506ef0                2                0               64   
C_ID_0001793786              125               76               15   
C_ID_000183fdda               11                1              131   

                enc_category_2_4             ...              category_1  \
                             sum             ...                    mean   
card_id                                      ...                           
C_ID_00007093c1                0             ...                0.187919   
C_ID_0001238066                0             ...                0.016260   
C_ID_0001506ef0                0             ...                0.000000   
C_ID_0001793786                0             ...                0.009259   
C_ID_000183fdda                0             ...                0.027778   

                                                           enc_purchase_date  \
                count_category_1_one count_category_1_zero              mean   
card_id                                                                        
C_ID_00007093c1                   28                   121      1.503054e+09   
C_ID_0001238066                    2                   121      1.513977e+09   
C_ID_0001506ef0                    0                    66      1.505646e+09   
C_ID_0001793786                    2                   214      1.499412e+09   
C_ID_000183fdda                    4                   140      1.512119e+09   

                                                         \
                          min           max         ptp   
card_id                                                   
C_ID_00007093c1  1.487081e+09  1.519708e+09  32627654.0   
C_ID_0001238066  1.506638e+09  1.519748e+09  13110825.0   
C_ID_0001506ef0  1.484411e+09  1.518871e+09  34460275.0   
C_ID_0001793786  1.484994e+09  1.509481e+09  24487497.0   
C_ID_000183fdda  1.502099e+09  1.519592e+09  17492874.0   

                               city_id                 state_id  \
                count_of_unique_cities count_of_unique_state_id   
card_id                                                           
C_ID_00007093c1                      4                        3   
C_ID_0001238066                     17                        5   
C_ID_0001506ef0                      3                        2   
C_ID_0001793786                      9                        3   
C_ID_000183fdda                      8                        6   

                                subsector_id  
                count_of_unique_subsector_id  
card_id                                       
C_ID_00007093c1                           13  
C_ID_0001238066                           17  
C_ID_0001506ef0                           12  
C_ID_0001793786                           24  
C_ID_000183fdda                           21  

[5 rows x 30 columns]

In [18]:
t_cols=['card_id']
t_cols.extend(['_'.join(i) for i in t.columns.tolist()])

t_cols

['card_id',
 'purchase_amount_sum',
 'purchase_amount_mean',
 'purchase_amount_max',
 'month_lag_sum',
 'month_lag_mean',
 'month_lag_max',
 'enc_category_2_1_sum',
 'enc_category_2_2_sum',
 'enc_category_2_3_sum',
 'enc_category_2_4_sum',
 'enc_category_2_5_sum',
 'enc_category_3_A_sum',
 'enc_category_3_B_sum',
 'enc_category_3_C_sum',
 'installments_sum',
 'installments_mean',
 'installments_max',
 'authorized_flag_mean',
 'authorized_flag_count_authorized_one',
 'authorized_flag_count_authorized_zero',
 'category_1_mean',
 'category_1_count_category_1_one',
 'category_1_count_category_1_zero',
 'enc_purchase_date_mean',
 'enc_purchase_date_min',
 'enc_purchase_date_max',
 'enc_purchase_date_ptp',
 'city_id_count_of_unique_cities',
 'state_id_count_of_unique_state_id',
 'subsector_id_count_of_unique_subsector_id']

In [19]:
t.reset_index(inplace=True)
t.columns = ['old_'+ i for i in t_cols]
t.head()

,old_card_id,old_purchase_amount_sum,old_purchase_amount_mean,old_purchase_amount_max,old_month_lag_sum,old_month_lag_mean,old_month_lag_max,old_enc_category_2_1_sum,old_enc_category_2_2_sum,old_enc_category_2_3_sum,...,old_category_1_mean,old_category_1_count_category_1_one,old_category_1_count_category_1_zero,old_enc_purchase_date_mean,old_enc_purchase_date_min,old_enc_purchase_date_max,old_enc_purchase_date_ptp,old_city_id_count_of_unique_cities,old_state_id_count_of_unique_state_id,old_subsector_id_count_of_unique_subsector_id
0,C_ID_00007093c1,-76.845041,-0.515739,1.507069,-872,-5.852349,0,28,0,120,...,0.187919,28,121,1.503054e+09,1.487081e+09,1.519708e+09,32627654.0,4,3,13
1,C_ID_0001238066,-72.447201,-0.589002,0.768095,-223,-1.813008,0,103,0,0,...,0.016260,2,121,1.513977e+09,1.506638e+09,1.519748e+09,13110825.0,17,5,17
2,C_ID_0001506ef0,-34.601879,-0.524271,1.493545,-319,-4.833333,0,2,0,64,...,0.000000,0,66,1.505646e+09,1.484411e+09,1.518871e+09,34460275.0,3,2,12
3,C_ID_0001793786,-36.786013,-0.170306,4.554145,-719,-3.328704,0,125,76,15,...,0.009259,2,214,1.499412e+09,1.484994e+09,1.509481e+09,24487497.0,9,3,24
4,C_ID_000183fdda,-68.837938,-0.478041,2.764788,-353,-2.451389,0,11,1,131,...,0.027778,4,140,1.512119e+09,1.502099e+09,1.519592e+09,17492874.0,8,6,21


In [20]:
t.rename(columns = {'old_card_id':'card_id'}, inplace=True)
t.head()

,card_id,old_purchase_amount_sum,old_purchase_amount_mean,old_purchase_amount_max,old_month_lag_sum,old_month_lag_mean,old_month_lag_max,old_enc_category_2_1_sum,old_enc_category_2_2_sum,old_enc_category_2_3_sum,...,old_category_1_mean,old_category_1_count_category_1_one,old_category_1_count_category_1_zero,old_enc_purchase_date_mean,old_enc_purchase_date_min,old_enc_purchase_date_max,old_enc_purchase_date_ptp,old_city_id_count_of_unique_cities,old_state_id_count_of_unique_state_id,old_subsector_id_count_of_unique_subsector_id
0,C_ID_00007093c1,-76.845041,-0.515739,1.507069,-872,-5.852349,0,28,0,120,...,0.187919,28,121,1.503054e+09,1.487081e+09,1.519708e+09,32627654.0,4,3,13
1,C_ID_0001238066,-72.447201,-0.589002,0.768095,-223,-1.813008,0,103,0,0,...,0.016260,2,121,1.513977e+09,1.506638e+09,1.519748e+09,13110825.0,17,5,17
2,C_ID_0001506ef0,-34.601879,-0.524271,1.493545,-319,-4.833333,0,2,0,64,...,0.000000,0,66,1.505646e+09,1.484411e+09,1.518871e+09,34460275.0,3,2,12
3,C_ID_0001793786,-36.786013,-0.170306,4.554145,-719,-3.328704,0,125,76,15,...,0.009259,2,214,1.499412e+09,1.484994e+09,1.509481e+09,24487497.0,9,3,24
4,C_ID_000183fdda,-68.837938,-0.478041,2.764788,-353,-2.451389,0,11,1,131,...,0.027778,4,140,1.512119e+09,1.502099e+09,1.519592e+09,17492874.0,8,6,21


In [21]:
t.shape, new_transaction_data.shape

((325540, 31), (29112361, 20))

In [22]:
new_transaction_data[['merchant_category_id','state_id','city_id','subsector_id','category_1']].isnull().any()

merchant_category_id    False
state_id                False
city_id                 False
subsector_id            False
category_1              False
dtype: bool

In [23]:
nn = new_transaction_data[['merchant_category_id','state_id','city_id','subsector_id','category_1','card_id']]
tt=panda.concat([t,nn], join='inner',axis = 1)
tt.shape

(325540, 37)

In [25]:
tt[['merchant_category_id','state_id','city_id','subsector_id','category_1','card_id']].head()

,merchant_category_id,state_id,city_id,subsector_id,category_1,card_id,card_id
0,80,16,88,37,0,C_ID_00007093c1,C_ID_4e6213e9bc
1,367,16,88,16,0,C_ID_0001238066,C_ID_4e6213e9bc
2,80,16,88,37,0,C_ID_0001506ef0,C_ID_4e6213e9bc
3,560,16,88,34,0,C_ID_0001793786,C_ID_4e6213e9bc
4,80,16,88,37,0,C_ID_000183fdda,C_ID_4e6213e9bc


In [26]:

import random

tt.columns = list(map(lambda x : x+'_'+str(random.randint(1,7)) if x=='card_id'else x, tt.columns.tolist()))

In [27]:
[i for i in tt.columns.tolist() if i.startswith('card_id')]

['card_id_3', 'card_id_4']

In [28]:
# tt[['merchant_category_id','state_id','city_id','subsector_id','category_1','card_id']].columns = ['merchant_category_id','state_id','city_id','subsector_id','category_1','card_id','_card_id']
tt.drop(columns = ['card_id_3'], inplace=True)
tt.rename(columns = {'card_id_4':'card_id'}, inplace =True)
tt[['merchant_category_id','state_id','city_id','subsector_id','category_1','card_id']].head()

,merchant_category_id,state_id,city_id,subsector_id,category_1,card_id
0,80,16,88,37,0,C_ID_4e6213e9bc
1,367,16,88,16,0,C_ID_4e6213e9bc
2,80,16,88,37,0,C_ID_4e6213e9bc
3,560,16,88,34,0,C_ID_4e6213e9bc
4,80,16,88,37,0,C_ID_4e6213e9bc


In [29]:
tt['category_1'] = tt.category_1.apply(lambda x: 'Y' if x==1 else 'N')
tt[['merchant_category_id','state_id','city_id','subsector_id','category_1','card_id']].head()

,merchant_category_id,state_id,city_id,subsector_id,category_1,card_id
0,80,16,88,37,N,C_ID_4e6213e9bc
1,367,16,88,16,N,C_ID_4e6213e9bc
2,80,16,88,37,N,C_ID_4e6213e9bc
3,560,16,88,34,N,C_ID_4e6213e9bc
4,80,16,88,37,N,C_ID_4e6213e9bc


In [30]:
## at this point we have 2 tables

## tt which is the aggregated data on new transaction data, aggregated by card id

## merchants data which is the merchants data aggregated by 'merchant_category_id','state_id','city_id','subsector_id','category_1'

## we are going to join the above two, to include merchant data on new transaction data

tt.shape, merchant_data.shape

((325540, 36), (46421, 66))

In [31]:
panda.concat([tt,merchant_data], join='inner', axis = 1).shape

(46421, 102)

In [32]:
temp_table = panda.merge(tt, merchant_data, how='left', on =['merchant_category_id','state_id','city_id','subsector_id','category_1'])

In [33]:
temp_table.dropna(inplace = True)
temp_table.isnull().any().sum()

0

In [34]:
temp_table.shape

(257389, 97)

In [35]:
temp_table.to_csv('all_new/old_trans_merchant_data.csv', index=False)